In [1]:
import json # read from json file
import re
import sklearn
from sklearn.utils import shuffle
import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics
import scipy.stats
import eli5
import pandas as pd

#with open('data.jsonl') as jsonl_file: # not using the full (unannotated) dataset in the below
#    lines = jsonl_file.readlines()
#jsonS = [json.loads(line) for line in lines]
#print(len(jsonS))

In [2]:
### RUN
with open('annotations2.jsonl') as jsonl_file:
    # note: after running data-preprocessing.ipynb this file already has token-level labels
    lines = jsonl_file.readlines()
annot = [json.loads(line) for line in lines]
#print(annot[0])

Chose whether or not to add leading "B-" / "I-" to class labels

In [22]:
########## ONLY RUN IF WE WANT TO ADD LEADING "B-" / "I-" TO CLASS LABEL
# now use above code and loop through all items of annot list:
# addLeading=1 for "Yes" (i.e. add leading "B-","I-" to annot); 0 for "No" (i.e. add labels to annot simply as they are)
addLeading = 1

if addLeading == 1:
    for j in range(0,len(annot)):
        a = annot[j]
        # select list of dict of tokens w/ annnotations and add column w/ no. of words to each dict:
        b = a['spans']
        # add noWords to b dict. note: b is list of dicts w/ annotations; tokens not on this list don't have annotations
        if b!=[]: #i.e. only try to add annotations to tokens if there are annotations to begin with
            #print(b)
            for i in range(0,len(annot[j]['tokens'])):
                    # now break-up label into 1st occurrence (leading "B-") and subsequent occurrences (leading "I-") (only for non "O"'s)
                    if annot[j]['tokens'][i]['label'] != "O":
                        if i==0:
                            annot[j]['tokens'][i]['label'] = "B-" + annot[j]['tokens'][i]['label']
                        else: 
                            if annot[j]['tokens'][i]['label'] == annot[j]['tokens'][i-1]['label'][2:]: # need to remove the leading "B-" that we had already been added to c[i-1]
                                annot[j]['tokens'][i]['label'] = "I-" + annot[j]['tokens'][i]['label']
                            else:
                                annot[j]['tokens'][i]['label'] = "B-" + annot[j]['tokens'][i]['label'] 

In [3]:
# now convert annotation tokens into list (sentences) of lists (tokens) format for sklearn_crfsuite.CRF
train_sents=[] 
for j in range(0,len(annot)):
    a = annot[j]['tokens']
    train_sentence = []
    for i in range(0,len(a)):
        if 'label' in a[i]: # only add element if this sample sentence has been labelled 
            token_element = (a[i]['text'],a[i]['label'])
            train_sentence.append(token_element)
    train_sents.append(train_sentence)

In [4]:
#X = [sent2features(s) for s in train_sents] # both are 140 x # of tokens in each example
#y = [sent2labels(s) for s in train_sents]
X = [[x for x,_ in s] for s in train_sents] # both are 140 x # of tokens in each example
y = [[y for _,y in s] for s in train_sents]

In [50]:
train_ratio = 0.75
train_test_split = round(0.75*len(X) - 0.5) # -0.5 => floor

# next 3 lines guarantee we always use same random indices for train / test set
idx_shuffle=pd.read_csv('idx_shuffle.csv') 
auxIdxList = idx_shuffle.values.tolist()
idx_shuffle = [el for auxList in auxIdxList for el in auxList]
X_shuffle, y_shuffle = [X[auxIdx] for auxIdx in idx_shuffle], [y[auxIdx] for auxIdx in idx_shuffle]
X_train, X_test, y_train, y_test = X_shuffle[:train_test_split], X_shuffle[train_test_split:], y_shuffle[:train_test_split], y_shuffle[train_test_split:]

In [49]:
# CODE TO BUILD CSV FILE WITH RANDOM INDICES
#idx = [i for i in range(0,len(X))]
#idx_shuffle = shuffle(idx,random_state=0)
#idx_shuffle_pandas = pd.DataFrame(idx_shuffle)
#idx_shuffle_pandas.to_csv('idx_shuffle.csv',index=False,header=True)

In [6]:
%time
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=True
)
crf.fit(X_train, y_train)

Wall time: 0 ns


C:\Users\vasco\anaconda3\envs\xcs224u\lib\site-packages\sklearn\base.py:209: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  warnings.warn('From version 0.24, get_params will raise an '


CRF(algorithm='lbfgs', all_possible_transitions=True, c1=0.1, c2=0.1,
    keep_tempfiles=None, max_iterations=100)

In [7]:
labels = list(crf.classes_)
y_pred = crf.predict(X_test)
print(metrics.flat_f1_score(y_test, y_pred,average='macro', labels=labels))
print(metrics.sequence_accuracy_score(y_test, y_pred))

0.6607080329881311
0.1111111111111111


In [8]:
# group B and I results
sorted_labels = sorted(
    labels,
    key=lambda name: (name[1:], name[0])
)
print(metrics.flat_classification_report(
    y_test, y_pred, labels=sorted_labels, digits=3
))

                     precision    recall  f1-score   support

                  O      0.938     0.973     0.955      1472
            KAEUFER      0.350     0.212     0.264        33
DATUM_VERBUECHERUNG      0.947     0.931     0.939        58
      DATUM_VERTRAG      0.857     0.844     0.850        64
         VERKAEUFER      0.578     0.388     0.464        67
   TERRASSENGROESSE      0.600     0.333     0.429         9
        GESAMTPREIS      1.000     0.759     0.863        29
            FLAECHE      0.651     0.737     0.691        38
           IMMO_TYP      0.469     0.460     0.465        50
            QMPREIS      0.682     0.652     0.667        23
                ORT      0.817     0.803     0.810        61
            STRASSE      0.600     0.477     0.532        44

           accuracy                          0.890      1948
          macro avg      0.707     0.631     0.661      1948
       weighted avg      0.880     0.890     0.883      1948



C:\Users\vasco\anaconda3\envs\xcs224u\lib\site-packages\sklearn\utils\validation.py:67: FutureWarning: Pass labels=['O', 'KAEUFER', 'DATUM_VERBUECHERUNG', 'DATUM_VERTRAG', 'VERKAEUFER', 'TERRASSENGROESSE', 'GESAMTPREIS', 'FLAECHE', 'IMMO_TYP', 'QMPREIS', 'ORT', 'STRASSE'] as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "


In [9]:
####### RUN THIS ONE
eli5.show_weights(crf, top=30)

In [10]:
from collections import Counter

def print_transitions(trans_features):
    for (label_from, label_to), weight in trans_features:
        print("%-6s -> %-7s %0.6f" % (label_from, label_to, weight))

print("Top likely transitions:")
print_transitions(Counter(crf.transition_features_).most_common(20))

print("\nTop unlikely transitions:")
print_transitions(Counter(crf.transition_features_).most_common()[-20:])

Top likely transitions:
O      -> O       5.966850
ORT    -> O       5.098497
FLAECHE -> O       4.806474
DATUM_VERTRAG -> O       4.678719
VERKAEUFER -> VERKAEUFER 4.388549
KAEUFER -> KAEUFER 4.290499
GESAMTPREIS -> O       4.246023
STRASSE -> STRASSE 4.225366
STRASSE -> O       3.810712
TERRASSENGROESSE -> O       3.598456
DATUM_VERBUECHERUNG -> DATUM_VERBUECHERUNG 3.547538
DATUM_VERTRAG -> DATUM_VERTRAG 3.219502
DATUM_VERBUECHERUNG -> O       3.086295
O      -> FLAECHE 2.936379
IMMO_TYP -> IMMO_TYP 2.648503
QMPREIS -> O       2.586461
IMMO_TYP -> O       2.493464
KAEUFER -> O       2.221254
VERKAEUFER -> O       2.194206
O      -> TERRASSENGROESSE 2.145914

Top unlikely transitions:
ORT    -> KAEUFER -2.276130
IMMO_TYP -> DATUM_VERBUECHERUNG -2.299294
VERKAEUFER -> GESAMTPREIS -2.426027
KAEUFER -> ORT     -2.462662
GESAMTPREIS -> DATUM_VERTRAG -2.494505
DATUM_VERBUECHERUNG -> GESAMTPREIS -2.506336
ORT    -> ORT     -2.538064
QMPREIS -> IMMO_TYP -2.565518
STRASSE -> DATUM_VERTRAG -2.